# Simple sentiment analysis techniques in python

This notebook is a guide to using the different techinques availble for sentiment analysis without training any data. Primarily these techniques only classify sentiments into positive and negative.  

In [ ]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import numpy as np 

## Dataset

We use some restaurant reviews as a small dataset. 

In [116]:
dataset=["Food is good and not too expensive. Serving is just right. Ambience is nice too.",
        "Fast service. Prices are reasonable and food is decent.",
        "Loved the ambience, loved the food",
        "Mushroom fried rice was tasty",
        "Service - Little slow, probably because too many people.",
        "The place is not easy to locate",
        "Extremely long waiting time. Food is decent but definitely not worth the wait.",
        "Used to be good. Soup was below average this time."]

Next we remove any non alphanumeric symbols. 

In [117]:
def tosentences(articles):
    tokenizer = RegexpTokenizer(r'[\w\?\.\!\'-]+')
    tokens=[]
    text=""
    
    tokens=(tokenizer.tokenize(articles))
    text=(" ".join(tokens))
    return text

In [118]:
query=[]
for i in dataset:
    query.append(tosentences(i))
               

In [119]:
query

['Food is good and not too expensive. Serving is just right. Ambience is nice too.',
 'Fast service. Prices are reasonable and food is decent.',
 'Loved the ambience loved the food',
 'Mushroom fried rice was tasty',
 'Service - Little slow probably because too many people.',
 'The place is not easy to locate',
 'Extremely long waiting time. Food is decent but definitely not worth the wait.',
 'Used to be good. Soup was below average this time.']

# Using IBM Watson Tone Analyser API

First we use the IBM Watson Tone Analyser API. To use this API, create a free account on IBM Cloud. Subscribe to the IBM Tone Analyser API. We are ready to use it for sentiment analysis now. This API is able to detect several emotions like happy,sad,joy,anger,fear,confidence and many more.
Also, it gives specific tone name and score to each sentence in a given piece of text.

In [120]:
from watson_developer_cloud import ToneAnalyzerV3
import watson_developer_cloud 
import json

In [121]:
tone_analyzer = ToneAnalyzerV3(
    version='2018-08-01',
    iam_apikey='KfDInKWxIi8O5nX4eRyX5UiWsQQLnt4xh7MQPy8hKfZu',
    url='https://gateway-wdc.watsonplatform.net/tone-analyzer/api'
)

In [122]:
def IBMToneAnalyse(dataquery):
    tone_analyzer = ToneAnalyzerV3(
    version='2018-08-01',
    iam_apikey='KfDInKWxIi8O5nX4eRyX5UiWsQQLnt4xh7MQPy8hKfZu',
    url='https://gateway-wdc.watsonplatform.net/tone-analyzer/api'
)
    
    try:
        import http.client as http_client
    except ImportError:
        import httplib as http_client
        http_client.HTTPConnection.debuglevel = 1
     
    json_output = tone_analyzer.tone(dataquery, content_type='text/plain')
    Dict_Tone=json_output.get_result()

    lst=[]
    flag=0

    for i in Dict_Tone['document_tone']['tones']:
        if (i):
            lst.append(["Tone Name : %s"%(i['tone_name']),"Score : %s" %(i['score']),"Tone Id : %s"%(i["tone_id"])])
            flag=1
        else:
            print "Error: Tone could not be recognised"
    
    if flag==1:
        return lst
    else:
        return 0

    

In [123]:
IBMsentiment={}
for i in dataset:
    IBMsentiment[i]=IBMToneAnalyse(i)

In [124]:
IBMsentiment

{'Extremely long waiting time. Food is decent but definitely not worth the wait.': [[u'Tone Name : Confident',
   'Score : 0.984417',
   u'Tone Id : confident']],
 'Fast service. Prices are reasonable and food is decent.': [[u'Tone Name : Analytical',
   'Score : 0.883404',
   u'Tone Id : analytical']],
 'Food is good and not too expensive. Serving is just right. Ambience is nice too.': [[u'Tone Name : Joy',
   'Score : 0.7559',
   u'Tone Id : joy'],
  [u'Tone Name : Tentative', 'Score : 0.944845', u'Tone Id : tentative']],
 'Loved the ambience, loved the food': [[u'Tone Name : Joy',
   'Score : 0.937302',
   u'Tone Id : joy']],
 'Mushroom fried rice was tasty': 0,
 'Service - Little slow, probably because too many people.': [[u'Tone Name : Sadness',
   'Score : 0.709872',
   u'Tone Id : sadness'],
  [u'Tone Name : Tentative', 'Score : 0.88939', u'Tone Id : tentative'],
  [u'Tone Name : Analytical', 'Score : 0.762356', u'Tone Id : analytical']],
 'The place is not easy to locate': 0,
 

# Using TextBlob pattern analyser


Next we have the TextBlob python library that is popular for text processing. It has a function for sentiment analysis where it gives the polarity and subjectivity of the concerned text. Negative polarity indicates negative sentiment and positive polarity indicates positive sentiment with the associated text. 
It has two implementations for analysing sentiments. One is PatternAnalyzer(default) which uses the pattern library and the other is the NaiveBayesAnalyser which is based on Naive Bayes algorithm.

In [125]:
from textblob import TextBlob

In [126]:
def TextblobToneAnalyse(dataquery):
    blob=TextBlob(dataquery)
    sentiment=blob.sentiment
    return [sentiment]

In [127]:
Textblobsentiment={}
for i in dataset:
    Textblobsentiment[i]=TextblobToneAnalyse(i)

In [128]:
Textblobsentiment

{'Extremely long waiting time. Food is decent but definitely not worth the wait.': [Sentiment(polarity=-0.011111111111111113, subjectivity=0.3888888888888889)],
 'Fast service. Prices are reasonable and food is decent.': [Sentiment(polarity=0.18888888888888888, subjectivity=0.6222222222222222)],
 'Food is good and not too expensive. Serving is just right. Ambience is nice too.': [Sentiment(polarity=0.2714285714285714, subjectivity=0.7089285714285715)],
 'Loved the ambience, loved the food': [Sentiment(polarity=0.7, subjectivity=0.8)],
 'Mushroom fried rice was tasty': [Sentiment(polarity=0.0, subjectivity=0.0)],
 'Service - Little slow, probably because too many people.': [Sentiment(polarity=0.004166666666666652, subjectivity=0.4666666666666666)],
 'The place is not easy to locate': [Sentiment(polarity=-0.21666666666666667, subjectivity=0.8333333333333334)],
 'Used to be good. Soup was below average this time.': [Sentiment(polarity=0.27499999999999997, subjectivity=0.5)]}

# Using TextBlob Naive Bayes analyser

The Naive Bayes TextBlob analyser classifies the sentiment associated with a sentence as positive, negative. It also gives a probabilistic value for the positive as well as negative sentiment associated with the sentence. 

In [129]:
from textblob.sentiments import NaiveBayesAnalyzer

In [130]:
def TextblobNaiveToneAnalyse(dataquery):
    blob=TextBlob(dataquery, analyzer=NaiveBayesAnalyzer())
    sentiment=blob.sentiment

    return [sentiment]

In [131]:
TextblobNaivesentiment={}
for i in dataset:
    TextblobNaivesentiment[i]=TextblobNaiveToneAnalyse(i)

In [132]:
TextblobNaivesentiment

{'Extremely long waiting time. Food is decent but definitely not worth the wait.': [Sentiment(classification='pos', p_pos=0.5614354038358297, p_neg=0.43856459616417026)],
 'Fast service. Prices are reasonable and food is decent.': [Sentiment(classification='neg', p_pos=0.3564904522147827, p_neg=0.6435095477852182)],
 'Food is good and not too expensive. Serving is just right. Ambience is nice too.': [Sentiment(classification='neg', p_pos=0.21266962938364903, p_neg=0.7873303706163495)],
 'Loved the ambience, loved the food': [Sentiment(classification='neg', p_pos=0.2808623117806499, p_neg=0.7191376882193501)],
 'Mushroom fried rice was tasty': [Sentiment(classification='neg', p_pos=0.41625088487702655, p_neg=0.5837491151229747)],
 'Service - Little slow, probably because too many people.': [Sentiment(classification='pos', p_pos=0.6535121616454963, p_neg=0.3464878383545031)],
 'The place is not easy to locate': [Sentiment(classification='pos', p_pos=0.6508420946901671, p_neg=0.3491579053

# Using Vader NLTK

The vader function from nltk library can be used for sentiment analysis. It takes a piece of text and gives the neutral, positive and negatve emotion associated with that text.

In [133]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/pranjali/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [134]:
def vaderToneAnalyse(sentence):
    nltkSentiment = SentimentIntensityAnalyzer()
    sentiment = nltkSentiment.polarity_scores(sentence)
    return [sentiment]

In [135]:
Vadersentiment={}
for i in dataset:
    Vadersentiment[i]=vaderToneAnalyse(i)
    

In [136]:
Vadersentiment

{'Extremely long waiting time. Food is decent but definitely not worth the wait.': [{'compound': 0.3718,
   'neg': 0.121,
   'neu': 0.665,
   'pos': 0.215}],
 'Fast service. Prices are reasonable and food is decent.': [{'compound': 0.0,
   'neg': 0.0,
   'neu': 1.0,
   'pos': 0.0}],
 'Food is good and not too expensive. Serving is just right. Ambience is nice too.': [{'compound': 0.6908,
   'neg': 0.0,
   'neu': 0.695,
   'pos': 0.305}],
 'Loved the ambience, loved the food': [{'compound': 0.8316,
   'neg': 0.0,
   'neu': 0.339,
   'pos': 0.661}],
 'Mushroom fried rice was tasty': [{'compound': 0.0,
   'neg': 0.0,
   'neu': 1.0,
   'pos': 0.0}],
 'Service - Little slow, probably because too many people.': [{'compound': 0.0,
   'neg': 0.0,
   'neu': 1.0,
   'pos': 0.0}],
 'The place is not easy to locate': [{'compound': -0.3412,
   'neg': 0.286,
   'neu': 0.714,
   'pos': 0.0}],
 'Used to be good. Soup was below average this time.': [{'compound': 0.4404,
   'neg': 0.0,
   'neu': 0.756,


# Azure API 

Microsoft hosted Azure API also supports sentiment analysis. However the authentication process is a little tedious. 

# Amazon Web Services API

AWS text analytics service - Amazon Comprehend also supports sentiment analysis. It requires to create an account with Amazon Web Services.